In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Constants
USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS_MONITORED = 950
TOTAL_URLS_UNMONITORED = 3000

# Load monitored data
print("Loading monitored datafile...")
with open("/content/drive/MyDrive/Colab Notebooks/dataset/mon_standard.pkl", 'rb') as fi:
    monitored_data = pickle.load(fi)

X1_mon, X2_mon, y_mon = [], [], []
for i in range(TOTAL_URLS_MONITORED):
    label = i // URL_PER_SITE
    for sample in monitored_data[i]:
        size_seq = []
        time_seq = []
        for c in sample:
            dr = 1 if c > 0 else -1
            time_seq.append(abs(c))
            size_seq.append(dr * 512)
        X1_mon.append(time_seq)
        X2_mon.append(size_seq)
        y_mon.append(label)

# Load unmonitored data
print("Loading unmonitored datafile...")
with open('/content/drive/MyDrive/Colab Notebooks/dataset/unmon_standard10_3000.pkl', 'rb') as f:
    unmonitored_data = pickle.load(f)

X1_unmon, X2_unmon, y_unmon = [], [], []
for i in range(TOTAL_URLS_UNMONITORED):
    size_seq = []
    time_seq = []
    for c in unmonitored_data[i]:
        dr = 1 if c > 0 else -1
        time_seq.append(abs(c))
        size_seq.append(dr * 512)
    X1_unmon.append(time_seq)
    X2_unmon.append(size_seq)
    y_unmon.append(-1)

# Combine monitored and unmonitored data
df_mon = pd.DataFrame({
    'time_seq': X1_mon,
    'size_seq': X2_mon,
    'label': y_mon
})
df_unmon = pd.DataFrame({
    'time_seq': X1_unmon,
    'size_seq': X2_unmon,
    'label': y_unmon
})

df_combined = pd.concat([df_mon, df_unmon], ignore_index=True)

# Extract Continuous Features
df_combined['cumulative_size'] = df_combined['size_seq'].apply(np.cumsum)
df_combined['burst_std'] = df_combined['size_seq'].apply(lambda x: np.std(np.diff(x, prepend=0)))
df_combined['mean_packet_size'] = df_combined['size_seq'].apply(np.mean)
df_combined['mean_timestamp'] = df_combined['time_seq'].apply(np.mean)

# Extract Categorical Features
df_combined['num_incoming'] = df_combined['size_seq'].apply(lambda x: sum(1 for i in x if i < 0))
df_combined['num_outgoing'] = df_combined['size_seq'].apply(lambda x: sum(1 for i in x if i > 0))
df_combined['fraction_incoming'] = df_combined['num_incoming'] / (df_combined['num_incoming'] + df_combined['num_outgoing'])
df_combined['fraction_outgoing'] = df_combined['num_outgoing'] / (df_combined['num_incoming'] + df_combined['num_outgoing'])
df_combined['total_packets'] = df_combined['num_incoming'] + df_combined['num_outgoing']

# Add Features
df_combined['packetsize_std'] = df_combined['size_seq'].apply(lambda x: np.std(x))
df_combined['timestamp_std'] = df_combined['time_seq'].apply(lambda x: np.std(x))
df_combined['mean_outgoing_packets'] = df_combined['size_seq'].apply(lambda x: np.mean([i for i in x if i > 0]) if any(i > 0 for i in x) else 0)
df_combined['packet_concentration_ordering'] = df_combined['size_seq'].apply(lambda x: np.mean(np.diff(x)))

# Fill NaN values (if any)
df_combined.fillna(0, inplace=True)

Loading monitored datafile...
Loading unmonitored datafile...


0. defense 적용 전: 61.95%

In [8]:
continuous_features = [
    'cumulative_size', 'burst_std', 'mean_packet_size', 'mean_timestamp',
    'packetsize_std', 'timestamp_std']
categorical_features = ['fraction_incoming', 'fraction_outgoing', 'total_packets', 'num_incoming', 'num_outgoing',
                        'mean_outgoing_packets', 'packet_concentration_ordering']

# Flatten lists
X_continuous = np.array(df_combined[continuous_features].map(lambda x: np.mean(x) if isinstance(x, np.ndarray) else x))
X_categorical = df_combined[categorical_features].values
X = np.hstack([X_continuous, X_categorical])
y = df_combined['label'].values

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize Continuous Features
scaler = StandardScaler()
X_train[:, :len(continuous_features)] = scaler.fit_transform(X_train[:, :len(continuous_features)])
X_test[:, :len(continuous_features)] = scaler.transform(X_test[:, :len(continuous_features)])

# 랜덤 포레스트 모델 정의
clf = RandomForestClassifier(random_state=123)

# 모델 훈련
clf.fit(X_train, y_train)

# 예측 수행
y_pred = clf.predict(X_test)

# 평가 지표 출력
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 61.95%
Classification Report:
              precision    recall  f1-score   support

          -1       0.43      0.53      0.48       882
           0       0.67      0.55      0.60        58
           1       0.46      0.43      0.44        60
           2       0.76      0.77      0.77        62
           3       0.65      0.65      0.65        46
           4       0.58      0.76      0.66        50
           5       0.86      0.82      0.84        60
           6       0.77      0.76      0.77        63
           7       0.49      0.70      0.58        50
           8       0.64      0.72      0.68        54
           9       0.53      0.58      0.55        48
          10       0.64      0.54      0.59        67
          11       0.64      0.59      0.62        64
          12       0.79      0.87      0.83        61
          13       0.54      0.28      0.37        76
          14       0.51      0.39      0.44        57
          15       0.67      0.79      0.

WF defense 적용

1. traffic split만 적용: 56.94%

In [11]:
df_combined = pd.concat([df_mon, df_unmon], ignore_index=True)

df_combined['cumulative_size'] = df_combined['size_seq'].apply(np.cumsum)
df_combined['burst_std'] = df_combined['size_seq'].apply(lambda x: np.std(np.diff(x, prepend=0)))
df_combined['mean_packet_size'] = df_combined['size_seq'].apply(np.mean)
df_combined['mean_timestamp'] = df_combined['time_seq'].apply(np.mean)
df_combined['num_incoming'] = df_combined['size_seq'].apply(lambda x: sum(1 for i in x if i < 0))
df_combined['num_outgoing'] = df_combined['size_seq'].apply(lambda x: sum(1 for i in x if i > 0))
df_combined['fraction_incoming'] = df_combined['num_incoming'] / (df_combined['num_incoming'] + df_combined['num_outgoing'])
df_combined['fraction_outgoing'] = df_combined['num_outgoing'] / (df_combined['num_incoming'] + df_combined['num_outgoing'])
df_combined['total_packets'] = df_combined['num_incoming'] + df_combined['num_outgoing']
df_combined['packetsize_std'] = df_combined['size_seq'].apply(lambda x: np.std(x))
df_combined['timestamp_std'] = df_combined['time_seq'].apply(lambda x: np.std(x))
df_combined['mean_outgoing_packets'] = df_combined['size_seq'].apply(lambda x: np.mean([i for i in x if i > 0]) if any(i > 0 for i in x) else 0)
df_combined['packet_concentration_ordering'] = df_combined['size_seq'].apply(lambda x: np.mean(np.diff(x)))

# Apply traffic splitting defense only
def traffic_split_only(df, max_splits=3):
    """Applying WF defense mechanisms: only traffic split..."""
    def defense_pipeline(row):
        split_size, split_time = split_traffic(row['size_seq'], row['time_seq'], max_splits)
        return split_size, split_time

    df['size_seq'], df['time_seq'] = zip(*df.apply(defense_pipeline, axis=1))
    return df

# Defining the helper defense function for traffic splitting
def split_traffic(size_seq, time_seq, max_splits=3):
    split_points = sorted(random.sample(range(1, len(size_seq)), random.randint(1, max_splits)))
    size_splits = [size_seq[i:j] for i, j in zip([0] + split_points, split_points + [len(size_seq)])]
    time_splits = [time_seq[i:j] for i, j in zip([0] + split_points, split_points + [len(time_seq)])]
    # Flatten and shuffle the segments to break sequence patterns
    shuffled_indices = list(range(len(size_splits)))
    random.shuffle(shuffled_indices)
    size_shuffled = [size for idx in shuffled_indices for size in size_splits[idx]]
    time_shuffled = [time for idx in shuffled_indices for time in time_splits[idx]]
    return size_shuffled, time_shuffled

# Apply the traffic split defense function to the combined dataset
df_combined = traffic_split_only(df_combined)

# Extract additional features
df_combined['cumulative_size'] = df_combined['size_seq'].apply(np.cumsum)
df_combined['burst_std'] = df_combined['size_seq'].apply(lambda x: np.std(np.diff(x, prepend=0)))
df_combined['mean_packet_size'] = df_combined['size_seq'].apply(np.mean)
df_combined['mean_timestamp'] = df_combined['time_seq'].apply(np.mean)
df_combined['num_incoming'] = df_combined['size_seq'].apply(lambda x: sum(1 for i in x if i < 0))
df_combined['num_outgoing'] = df_combined['size_seq'].apply(lambda x: sum(1 for i in x if i > 0))
df_combined['fraction_incoming'] = df_combined['num_incoming'] / (df_combined['num_incoming'] + df_combined['num_outgoing'])
df_combined['fraction_outgoing'] = df_combined['num_outgoing'] / (df_combined['num_incoming'] + df_combined['num_outgoing'])
df_combined['total_packets'] = df_combined['num_incoming'] + df_combined['num_outgoing']

df_combined['packetsize_std'] = df_combined['size_seq'].apply(lambda x: np.std(x))
df_combined['timestamp_std'] = df_combined['time_seq'].apply(lambda x: np.std(x))
df_combined['mean_outgoing_packets'] = df_combined['size_seq'].apply(lambda x: np.mean([i for i in x if i > 0]) if any(i > 0 for i in x) else 0)
df_combined['packet_concentration_ordering'] = df_combined['size_seq'].apply(lambda x: np.mean(np.diff(x)))

# Extract the features for training
continuous_features = [
    'cumulative_size', 'burst_std', 'mean_packet_size', 'mean_timestamp',
    'packetsize_std', 'timestamp_std']
categorical_features = ['fraction_incoming', 'fraction_outgoing', 'total_packets', 'num_incoming', 'num_outgoing',
                        'mean_outgoing_packets', 'packet_concentration_ordering']

X_continuous = np.array(df_combined[continuous_features].map(lambda x: np.mean(x) if isinstance(x, np.ndarray) else x))
X_categorical = df_combined[categorical_features].values
X = np.hstack([X_continuous, X_categorical])
y = df_combined['label'].values

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fill NaN values (if any)
df_combined.fillna(0, inplace=True)

# Standard scaling
scaler = StandardScaler()
X_train[:, :len(continuous_features)] = scaler.fit_transform(X_train[:, :len(continuous_features)])
X_test[:, :len(continuous_features)] = scaler.transform(X_test[:, :len(continuous_features)])

# 랜덤 포레스트 모델 정의
clf = RandomForestClassifier(random_state=123)

# 모델 훈련
clf.fit(X_train, y_train)

# 예측 수행
y_pred = clf.predict(X_test)

# 평가 지표 출력
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 56.94%
Classification Report:
              precision    recall  f1-score   support

          -1       0.42      0.50      0.46       882
           0       0.52      0.45      0.48        58
           1       0.40      0.35      0.38        60
           2       0.69      0.73      0.71        62
           3       0.49      0.52      0.51        46
           4       0.48      0.68      0.56        50
           5       0.73      0.77      0.75        60
           6       0.75      0.68      0.72        63
           7       0.49      0.68      0.57        50
           8       0.58      0.57      0.58        54
           9       0.46      0.50      0.48        48
          10       0.59      0.45      0.51        67
          11       0.67      0.55      0.60        64
          12       0.73      0.87      0.79        61
          13       0.47      0.25      0.33        76
          14       0.44      0.37      0.40        57
          15       0.59      0.77      0.

2. padding, random delay, traffic split만 적용: 52.95%

In [9]:
import random

# WF Defense mechanisms
def apply_padding(size_seq, max_padding=512):
    """Apply random padding to packet sizes."""
    return [s + random.randint(-max_padding, max_padding) for s in size_seq]

def insert_random_delays(time_seq, max_delay=100):
    """Insert random delays into time sequence."""
    return [t + random.uniform(0, max_delay) for t in time_seq]

def split_traffic(size_seq, time_seq, max_splits=3):
    """Randomly split traffic into multiple smaller segments."""
    split_points = sorted(random.sample(range(1, len(size_seq)), random.randint(1, max_splits)))
    size_splits = [size_seq[i:j] for i, j in zip([0] + split_points, split_points + [len(size_seq)])]
    time_splits = [time_seq[i:j] for i, j in zip([0] + split_points, split_points + [len(time_seq)])]
    # Flatten and shuffle the segments to break sequence patterns
    shuffled_indices = list(range(len(size_splits)))
    random.shuffle(shuffled_indices)
    size_shuffled = [size for idx in shuffled_indices for size in size_splits[idx]]
    time_shuffled = [time for idx in shuffled_indices for time in time_splits[idx]]
    return size_shuffled, time_shuffled

def perturb_data_with_all_defenses(df, max_padding=512, max_delay=100, max_splits=3):
    """Apply all WF defenses: padding, delays, and traffic splitting."""
    df['size_seq'], df['time_seq'] = zip(*df.apply(
        lambda row: split_traffic(
            apply_padding(row['size_seq'], max_padding),
            insert_random_delays(row['time_seq'], max_delay),
            max_splits=max_splits
        ), axis=1))
    return df

# Apply all WF defenses to the combined dataset
print("Applying WF defense mechanisms: padding, delays, and traffic splitting...")
df_combined = perturb_data_with_all_defenses(df_combined, max_padding=256, max_delay=50, max_splits=3)

# Recompute features after applying all defenses
df_combined['cumulative_size'] = df_combined['size_seq'].apply(np.cumsum)
df_combined['burst_std'] = df_combined['size_seq'].apply(lambda x: np.std(np.diff(x, prepend=0)))
df_combined['mean_packet_size'] = df_combined['size_seq'].apply(np.mean)
df_combined['mean_timestamp'] = df_combined['time_seq'].apply(np.mean)
df_combined['num_incoming'] = df_combined['size_seq'].apply(lambda x: sum(1 for i in x if i < 0))
df_combined['num_outgoing'] = df_combined['size_seq'].apply(lambda x: sum(1 for i in x if i > 0))
df_combined['fraction_incoming'] = df_combined['num_incoming'] / (df_combined['num_incoming'] + df_combined['num_outgoing'])
df_combined['fraction_outgoing'] = df_combined['num_outgoing'] / (df_combined['num_incoming'] + df_combined['num_outgoing'])
df_combined['total_packets'] = df_combined['num_incoming'] + df_combined['num_outgoing']
df_combined['packetsize_std'] = df_combined['size_seq'].apply(lambda x: np.std(x))
df_combined['timestamp_std'] = df_combined['time_seq'].apply(lambda x: np.std(x))
df_combined['mean_outgoing_packets'] = df_combined['size_seq'].apply(lambda x: np.mean([i for i in x if i > 0]) if any(i > 0 for i in x) else 0)
df_combined['packet_concentration_ordering'] = df_combined['size_seq'].apply(lambda x: np.mean(np.diff(x)))

# Fill NaN values (if any)
df_combined.fillna(0, inplace=True)

# Extract the features again
X_continuous = np.array(df_combined[continuous_features].map(lambda x: np.mean(x) if isinstance(x, np.ndarray) else x))
X_categorical = df_combined[categorical_features].values
X = np.hstack([X_continuous, X_categorical])
y = df_combined['label'].values

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standard scaling
scaler = StandardScaler()
X_train[:, :len(continuous_features)] = scaler.fit_transform(X_train[:, :len(continuous_features)])
X_test[:, :len(continuous_features)] = scaler.transform(X_test[:, :len(continuous_features)])

# 랜덤 포레스트 모델 정의
clf = RandomForestClassifier(random_state=123)

# 모델 훈련
clf.fit(X_train, y_train)

# 예측 수행
y_pred = clf.predict(X_test)

# 평가 지표 출력
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Applying WF defense mechanisms: padding, delays, and traffic splitting...
Accuracy: 52.95%
Classification Report:
              precision    recall  f1-score   support

          -1       0.37      0.48      0.42       882
           0       0.47      0.33      0.39        58
           1       0.33      0.28      0.30        60
           2       0.68      0.77      0.72        62
           3       0.49      0.54      0.52        46
           4       0.44      0.66      0.53        50
           5       0.71      0.75      0.73        60
           6       0.71      0.59      0.64        63
           7       0.48      0.78      0.59        50
           8       0.62      0.54      0.57        54
           9       0.37      0.48      0.41        48
          10       0.57      0.42      0.48        67
          11       0.55      0.42      0.48        64
          12       0.71      0.87      0.78        61
          13       0.36      0.17      0.23        76
          14       0.

3. padding, random delay, traffic split, dummy_traffic , time_warping, uniform_packet_size 적용: 33.18%

In [10]:
# Apply enhanced defense with traffic splitting
def enhanced_defense_with_split(df, max_splits=3, uniform_size=512, dummy_prob=0.3, time_scale_range=(0.5, 1.5)):
    """Applying WF defense mechanisms: padding, random delay, traffic split, dummy_traffic , time_warping, uniform_packet_size..."""
    def defense_pipeline(row):
        # 1. 패킷 크기 균일화
        uniformed = uniform_packet_size(row['size_seq'], uniform_size)
        # 2. 더미 트래픽 삽입
        dummy_added = add_dummy_traffic(uniformed, dummy_prob)
        # 3. 트래픽 분할
        split_size, split_time = split_traffic(dummy_added, row['time_seq'], max_splits)
        # 4. 랜덤 시간 왜곡
        warped_time = time_warping(split_time, time_scale_range)
        return split_size, warped_time

    df['size_seq'], df['time_seq'] = zip(*df.apply(defense_pipeline, axis=1))
    return df

# Defining the helper defense functions

import random

def apply_padding(size_seq, max_padding=512):
    """Apply random padding to packet sizes."""
    return [s + random.randint(-max_padding, max_padding) for s in size_seq]

def insert_random_delays(time_seq, max_delay=100):
    """Insert random delays into time sequence."""
    return [t + random.uniform(0, max_delay) for t in time_seq]

def split_traffic(size_seq, time_seq, max_splits=3):
    """Randomly split traffic into multiple smaller segments."""
    split_points = sorted(random.sample(range(1, len(size_seq)), random.randint(1, max_splits)))
    size_splits = [size_seq[i:j] for i, j in zip([0] + split_points, split_points + [len(size_seq)])]
    time_splits = [time_seq[i:j] for i, j in zip([0] + split_points, split_points + [len(time_seq)])]
    # Flatten and shuffle the segments to break sequence patterns
    shuffled_indices = list(range(len(size_splits)))
    random.shuffle(shuffled_indices)
    size_shuffled = [size for idx in shuffled_indices for size in size_splits[idx]]
    time_shuffled = [time for idx in shuffled_indices for time in time_splits[idx]]
    return size_shuffled, time_shuffled

def add_dummy_traffic(size_seq, dummy_prob=0.3):
    """Insert dummy packets into the sequence."""
    result = []
    for size in size_seq:
        result.append(size)
        if random.random() < dummy_prob:
            result.append(random.choice([-512, 512]))
    return result

def uniform_packet_size(size_seq, uniform_size=512):
    """Normalize all packet sizes to a uniform value."""
    return [uniform_size if s > 0 else -uniform_size for s in size_seq]

def time_warping(time_seq, scale_factor_range=(0.5, 1.5)):
    """Apply random time warping to distort timestamp patterns."""
    scale_factor = random.uniform(*scale_factor_range)
    return [t * scale_factor for t in time_seq]

# Apply the enhanced defense function to the combined dataset
df_combined = enhanced_defense_with_split(df_combined)

# Extract additional features
df_combined['cumulative_size'] = df_combined['size_seq'].apply(np.cumsum)
df_combined['burst_std'] = df_combined['size_seq'].apply(lambda x: np.std(np.diff(x, prepend=0)))
df_combined['mean_packet_size'] = df_combined['size_seq'].apply(np.mean)
df_combined['mean_timestamp'] = df_combined['time_seq'].apply(np.mean)
df_combined['num_incoming'] = df_combined['size_seq'].apply(lambda x: sum(1 for i in x if i < 0))
df_combined['num_outgoing'] = df_combined['size_seq'].apply(lambda x: sum(1 for i in x if i > 0))
df_combined['fraction_incoming'] = df_combined['num_incoming'] / (df_combined['num_incoming'] + df_combined['num_outgoing'])
df_combined['fraction_outgoing'] = df_combined['num_outgoing'] / (df_combined['num_incoming'] + df_combined['num_outgoing'])
df_combined['total_packets'] = df_combined['num_incoming'] + df_combined['num_outgoing']
df_combined['packetsize_std'] = df_combined['size_seq'].apply(lambda x: np.std(x))
df_combined['timestamp_std'] = df_combined['time_seq'].apply(lambda x: np.std(x))
df_combined['mean_outgoing_packets'] = df_combined['size_seq'].apply(lambda x: np.mean([i for i in x if i > 0]) if any(i > 0 for i in x) else 0)
df_combined['packet_concentration_ordering'] = df_combined['size_seq'].apply(lambda x: np.mean(np.diff(x)))

# Extract the features for training
continuous_features = [
    'cumulative_size', 'burst_std', 'mean_packet_size', 'mean_timestamp',
    'packetsize_std', 'timestamp_std']
categorical_features = ['fraction_incoming', 'fraction_outgoing', 'total_packets', 'num_incoming', 'num_outgoing',
                        'mean_outgoing_packets', 'packet_concentration_ordering']

X_continuous = np.array(df_combined[continuous_features].map(lambda x: np.mean(x) if isinstance(x, np.ndarray) else x))
X_categorical = df_combined[categorical_features].values
X = np.hstack([X_continuous, X_categorical])
y = df_combined['label'].values

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fill NaN values (if any)
df_combined.fillna(0, inplace=True)

# Standard scaling
scaler = StandardScaler()
X_train[:, :len(continuous_features)] = scaler.fit_transform(X_train[:, :len(continuous_features)])
X_test[:, :len(continuous_features)] = scaler.transform(X_test[:, :len(continuous_features)])

# 랜덤 포레스트 모델 정의
clf = RandomForestClassifier(random_state=123)

# 모델 훈련
clf.fit(X_train, y_train)

# 예측 수행
y_pred = clf.predict(X_test)

# 평가 지표 출력
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 33.18%
Classification Report:
              precision    recall  f1-score   support

          -1       0.22      0.32      0.26       882
           0       0.31      0.19      0.24        58
           1       0.12      0.13      0.13        60
           2       0.45      0.63      0.52        62
           3       0.22      0.24      0.23        46
           4       0.20      0.36      0.25        50
           5       0.48      0.37      0.42        60
           6       0.37      0.44      0.40        63
           7       0.22      0.32      0.26        50
           8       0.48      0.43      0.45        54
           9       0.16      0.23      0.19        48
          10       0.46      0.27      0.34        67
          11       0.27      0.27      0.27        64
          12       0.63      0.67      0.65        61
          13       0.22      0.11      0.14        76
          14       0.37      0.26      0.31        57
          15       0.42      0.56      0.